In [1]:
#распаковка модулей DL-Go
!unzip -o dlgo.zip

Archive:  dlgo.zip
   creating: dlgo/
   creating: dlgo/agent/
  inflating: dlgo/agent/base.py      
  inflating: dlgo/agent/helpers.py   
  inflating: dlgo/agent/naive.py     
  inflating: dlgo/agent/predict.py   
  inflating: dlgo/agent/termination.py  
 extracting: dlgo/agent/__init__.py  
   creating: dlgo/agent/__pycache__/
  inflating: dlgo/agent/__pycache__/base.cpython-310.pyc  
  inflating: dlgo/agent/__pycache__/helpers.cpython-310.pyc  
  inflating: dlgo/agent/__pycache__/naive.cpython-310.pyc  
  inflating: dlgo/agent/__pycache__/predict.cpython-310.pyc  
  inflating: dlgo/agent/__pycache__/termination.cpython-310.pyc  
  inflating: dlgo/agent/__pycache__/__init__.cpython-310.pyc  
   creating: dlgo/data/
  inflating: dlgo/data/generator.py  
  inflating: dlgo/data/index_processor.py  
  inflating: dlgo/data/parallel_processor.py  
  inflating: dlgo/data/processor.py  
  inflating: dlgo/data/sampling.py   
  inflating: dlgo/data/__init__.py   
   creating: dlgo/data/__pycac

Загрузка признаков и меток с помощью обработчика данных

In [2]:
import h5py

from keras.models import Sequential
from keras.layers import Dense

from dlgo.agent.predict import DeepLearningAgent, load_prediction_agent
#from dlgo.data.parallel_processor import GoDataProcessor #Wall time: 2h 37min 57s
from dlgo.data.processor import GoDataProcessor
from dlgo.encoders.sevenplane import SevenPlaneEncoder
from dlgo.networks import large

from dlgo.httpfrontend import get_web_app

In [4]:
go_board_rows, go_board_cols = 19, 19
nb_classes = go_board_rows * go_board_cols
encoder = SevenPlaneEncoder((go_board_rows, go_board_cols))
processor = GoDataProcessor(encoder=encoder.name())

In [5]:
%%time

# данные берем из NPY собранного или формируем из SGF
npy_data = True

if not npy_data:
  X, y = processor.load_go_data(num_samples=100)
  !cp ./data/*.npy ./drive/MyDrive/colab1/
else:
  !mkdir ./data/
  !cp ./drive/MyDrive/colab1/features_train.npy ./data/
  !cp ./drive/MyDrive/colab1/labels_train.npy ./data/
  X, y = processor.load_go_data_npy()

CPU times: user 20.9 ms, sys: 81.7 ms, total: 103 ms
Wall time: 2.42 s


Создание и запуск модели для предсказания ходов с использованием оптимизатора Adadelta

In [4]:
%%time

input_shape = (encoder.num_planes, go_board_rows, go_board_cols)
model = Sequential()
network_layers = large.layers(input_shape)
for layer in network_layers:
    model.add(layer)
model.add(Dense(nb_classes, activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adadelta', metrics=['accuracy'])
model.fit(X, y, batch_size=128, epochs=20, verbose=1)

Epoch 1/20
88/88 [==============================] - 15s 29ms/step - loss: 5.8887 - accuracy: 0.0034
Epoch 2/20
88/88 [==============================] - 3s 29ms/step - loss: 5.8887 - accuracy: 0.0035
Epoch 3/20
88/88 [==============================] - 3s 29ms/step - loss: 5.8887 - accuracy: 0.0034
Epoch 4/20
88/88 [==============================] - 3s 30ms/step - loss: 5.8886 - accuracy: 0.0034
Epoch 5/20
88/88 [==============================] - 3s 30ms/step - loss: 5.8886 - accuracy: 0.0035
Epoch 6/20
88/88 [==============================] - 3s 29ms/step - loss: 5.8886 - accuracy: 0.0036
Epoch 7/20
88/88 [==============================] - 3s 29ms/step - loss: 5.8885 - accuracy: 0.0037
Epoch 8/20
88/88 [==============================] - 3s 29ms/step - loss: 5.8885 - accuracy: 0.0038
Epoch 9/20
88/88 [==============================] - 3s 30ms/step - loss: 5.8885 - accuracy: 0.0042
Epoch 10/20
88/88 [==============================] - 3s 30ms/step - loss: 5.8884 - accuracy: 0.0043
Epoch 11

Создание и сохранение агента DeepLearningAgent

In [5]:
deep_learning_bot = DeepLearningAgent(model, encoder)
h5file = h5py.File("./agents/sevenplane.h5", "w")
deep_learning_bot.serialize(h5file)
h5file.close()

In [6]:
!cp ./agents/sevenplane.h5 ./drive/MyDrive/colab1/sevenplane.h5

Загрузка бота обратно в память...

In [3]:
!mkdir ./agents/
!cp ./drive/MyDrive/colab1/sevenplane.h5 ./agents/sevenplane.h5

In [4]:
# должен быть: GPU?
model_file = h5py.File("./agents/sevenplane.h5", "r")
bot_from_file = load_prediction_agent(model_file)

...и его подключение к веб-приложению

In [ ]:
web_app = get_web_app({'predict': bot_from_file})
web_app.run()

 * Serving Flask app 'dlgo.httpfrontend.server'
 * Debug mode: off


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on http://127.0.0.1:5000
INFO:werkzeug:Press CTRL+C to quit
